# 02 - Tuning Parametri HSV

Notebook interattivo per ottimizzare i parametri di detection HSV.

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from pathlib import Path

%matplotlib inline

## 1. Carica Frame

In [ ]:
VIDEO_PATH = '../data/videos/input/video1.mp4'

cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
cap.release()

if ret:
    print(f"Frame caricato: {frame.shape}")
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
else:
    print("Errore caricamento")

## 2. Visualizza Canali HSV

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Originale (RGB)')
axes[0, 0].axis('off')

axes[0, 1].imshow(hsv_frame[:,:,0], cmap='hsv')
axes[0, 1].set_title('Hue (H)')
axes[0, 1].axis('off')

axes[1, 0].imshow(hsv_frame[:,:,1], cmap='gray')
axes[1, 0].set_title('Saturation (S)')
axes[1, 0].axis('off')

axes[1, 1].imshow(hsv_frame[:,:,2], cmap='gray')
axes[1, 1].set_title('Value (V)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nStatistiche HSV:")
print(f"  H range: [{hsv_frame[:,:,0].min()}, {hsv_frame[:,:,0].max()}]")
print(f"  S range: [{hsv_frame[:,:,1].min()}, {hsv_frame[:,:,1].max()}]")
print(f"  V range: [{hsv_frame[:,:,2].min()}, {hsv_frame[:,:,2].max()}]")

## 3. Tuning Interattivo Range Rosso

In [ ]:
def test_red_range(h_lower1, h_upper1, h_lower2, h_upper2, s_lower, v_lower):
    lower1 = np.array([h_lower1, s_lower, v_lower])
    upper1 = np.array([h_upper1, 255, 255])
    mask1 = cv2.inRange(hsv_frame, lower1, upper1)
    
    lower2 = np.array([h_lower2, s_lower, v_lower])
    upper2 = np.array([h_upper2, 255, 255])
    mask2 = cv2.inRange(hsv_frame, lower2, upper2)
    
    mask = cv2.bitwise_or(mask1, mask2)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Originale')
    axes[0].axis('off')
    
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title(f'Maschera (Pixel: {np.sum(mask > 0)})')
    axes[1].axis('off')
    
    overlay = frame.copy()
    overlay[mask > 0] = [0, 255, 0]
    result = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
    axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Range 1: H=[{h_lower1}, {h_upper1}], S>={s_lower}, V>={v_lower}")
    print(f"Range 2: H=[{h_lower2}, {h_upper2}], S>={s_lower}, V>={v_lower}")

interact(
    test_red_range,
    h_lower1=IntSlider(min=0, max=10, step=1, value=0, description='H Lower 1'),
    h_upper1=IntSlider(min=0, max=20, step=1, value=10, description='H Upper 1'),
    h_lower2=IntSlider(min=160, max=180, step=1, value=170, description='H Lower 2'),
    h_upper2=IntSlider(min=170, max=180, step=1, value=180, description='H Upper 2'),
    s_lower=IntSlider(min=0, max=255, step=5, value=100, description='S Lower'),
    v_lower=IntSlider(min=0, max=255, step=5, value=100, description='V Lower')
);